![](https://mcd.unison.mx/wp-content/themes/awaken/img/logo_mcd.png)

# Curso Ingeniería de Características

## Análisis en Componentes Principales

### Julio Waissman


En esta libreta vamos a hacer una pequeña aplicación del Análisis en Componentes Principales (PCA) aplicado a los datos de marginación del estado de Sonora, los cuales los vamos a consultar [desde la página del gobierno federal para el año 2020](https://www.gob.mx/conapo/documentos/indices-de-marginacion-2020-284372).

No vamos a utilizar los indices normalizados de marginación por municipio desarrollados por la CONAPO ya que lo que queremos es hacer un análisis exploratorio de datos multivariados utilizndo componentes principales.

### 1. Cargando y seleccionando los datos de marginación

Empecemos por cargar los datos y ver como son:

In [5]:
import pandas as pd

url = "http://www.conapo.gob.mx/work/models/CONAPO/Marginacion/Datos_Abiertos/Municipio/IMM_2020.xls"

dic_df = pd.read_excel(url, None)

dic_df.keys()

dict_keys(['Base de marginación 2020', 'IMM_2020', 'Diccionario'])

Nos vamos a quedar con sólo los municipios de sonora. En `IMM_2020` se encuentran los datos para fácil consumo de un programa, y en 'Diccionario' se encuentra la descripción de cada variable.

Vamos a ver en el diccionario de datos cuales son los datos que nos interesan.

In [6]:
dic_df['Diccionario']

,"Indicadores sociodemográficos e índice de marginación a nivel municipal, 2020",Unnamed: 1
0,NaN,NaN
1,Variable,Descripción
2,CVE_ENT,Clave de entidad federativa
3,NOM_ENT,Nombre de entidad federativa
4,CVE_MUN,Clave del municipio
5,NOM_MUN,Nombre del municipio
6,POB_TOT,Población total
7,ANALF,Porcentaje de población analfabeta de 15 años ...
8,SBASC,Porcentaje de población de 15 años o mas sin e...
9,OVSDE,Porcentaje de ocupantes en viviendas particula...


Vamos primero a separar los datos de sonora y revisar si se encuentran todos losmunicipios del estado:

In [42]:
sonora = dic_df['IMM_2020'][dic_df['IMM_2020'].NOM_ENT.str.contains('Sonora')]
print(f"Los municipios de Sonora son: \n {sonora.NOM_MUN.unique()}")
sonora

Los municipios de Sonora son: 
 ['Aconchi' 'Agua Prieta' 'Alamos' 'Altar' 'Arivechi' 'Arizpe' 'Atil'
 'Bacadéhuachi' 'Bacanora' 'Bacerac' 'Bacoachi' 'Bácum' 'Banámichi'
 'Baviácora' 'Bavispe' 'Benjamín Hill' 'Caborca' 'Cajeme' 'Cananea'
 'Carbó' 'La Colorada' 'Cucurpe' 'Cumpas' 'Divisaderos' 'Empalme'
 'Etchojoa' 'Fronteras' 'Granados' 'Guaymas' 'Hermosillo' 'Huachinera'
 'Huásabas' 'Huatabampo' 'Huépac' 'Imuris' 'Magdalena' 'Mazatán'
 'Moctezuma' 'Naco' 'Nácori Chico' 'Nacozari de García' 'Navojoa'
 'Nogales' 'Ónavas' 'Opodepe' 'Oquitoa' 'Pitiquito' 'Puerto Peñasco'
 'Quiriego' 'Rayón' 'Rosario' 'Sahuaripa' 'San Felipe de Jesús'
 'San Javier' 'San Luis Río Colorado' 'San Miguel de Horcasitas'
 'San Pedro de la Cueva' 'Santa Ana' 'Santa Cruz' 'Sáric' 'Soyopa'
 'Suaqui Grande' 'Tepache' 'Trincheras' 'Tubutama' 'Ures' 'Villa Hidalgo'
 'Villa Pesqueira' 'Yécora' 'General Plutarco Elías Calles'
 'Benito Juárez' 'San Ignacio Río Muerto']


,CVE_ENT,NOM_ENT,CVE_MUN,NOM_MUN,POB_TOT,ANALF,SBASC,OVSDE,OVSEE,OVSAE,OVPT,VHAC,PL.5000,PO2SM,IM_2020,GM_2020,IMN_2020
1901,26,Sonora,26001,Aconchi,2563,2.691722,37.125444,0.780336,0.858369,0.702302,0.312134,16.329113,100.000000,66.869301,57.135014,Muy bajo,0.895231
1902,26,Sonora,26002,Agua Prieta,91929,1.488203,26.019518,0.203682,1.035895,0.618724,0.946196,17.610458,0.979016,81.065659,58.591155,Muy bajo,0.918047
1903,26,Sonora,26003,Alamos,24976,8.107527,41.656634,7.845339,4.087117,5.190117,10.769724,26.759377,56.113869,75.732267,53.395465,Medio,0.836637
1904,26,Sonora,26004,Altar,9492,3.433923,35.089742,1.144674,0.932697,1.028087,1.017920,23.018867,11.093552,89.689880,57.213979,Muy bajo,0.896468
1905,26,Sonora,26005,Arivechi,1177,3.265766,45.912654,0.256630,3.250642,0.085543,1.881950,10.997442,100.000000,60.227273,56.691898,Muy bajo,0.888288
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1968,26,Sonora,26068,Villa Pesqueira,1043,2.441860,41.811847,3.464870,1.732435,1.924928,1.924928,7.200000,100.000000,70.891089,56.553727,Muy bajo,0.886123
1969,26,Sonora,26069,Yécora,4793,9.025788,51.576835,1.761376,3.564689,6.689033,12.350598,19.512196,100.000000,79.286336,53.146764,Medio,0.832740
1970,26,Sonora,26070,General Plutarco Elías Calles,13627,3.092565,31.458325,0.536633,0.767683,1.043452,1.281956,21.671598,16.049020,88.378327,57.492467,Muy bajo,0.900832
1971,26,Sonora,26071,Benito Juárez,21692,3.946547,30.309743,1.184472,1.064174,1.411188,1.813891,28.978703,39.364743,79.088972,56.813878,Muy bajo,0.890199


Como podemos ver tenemos los 72 municipio. Nos vamos entonces a quedar con los datos de las columnas 7 a 15 (porcentajes) sin tomar en cuenta el tamaño de la población (en forma conciente)

In [50]:
dic_datos = dic_df['Diccionario'].iloc[7:15,:].copy().rename(columns={
    "Indicadores sociodemográficos e índice de marginación a nivel municipal, 2020": "Indicador",
    "Unnamed: 1": "Descripcion" 
})

datos = sonora[dic_datos.Indicador.values].to_numpy()

# Vamos a ver, deberíamos tener 72 municipios, cada uno representado por 8 indicadores
datos.shape

(72, 8)

### 2. Aplicando el Análisis en componentes principales

Vamos a ver si podemos utilizar PCA para revisar estos datos que se encuentran en 8 dimensiones y ver si los datos de marginación tendrían un significado diferente si se analizan a nivel estatal. Así no se tendrían los sesgos de municipios con menor marginación como Garza García o San Nicolás de los Garza en Nuevo León, Cuauthemoc o Benito Juárez en la Ciudad de México, pero tampoco de los municipios con mayor marginación como Batopilas en Chihuahua, Mezquital en Durango, Del Nayar en Nayarit o Mezquitic en Jalisco.  

,CVE_ENT,NOM_ENT,CVE_MUN,NOM_MUN,POB_TOT,ANALF,SBASC,OVSDE,OVSEE,OVSAE,OVPT,VHAC,PL.5000,PO2SM,IM_2020,GM_2020,IMN_2020
213,8,Chihuahua,8008,Batopilas de Manuel Gómez Morín,11270,45.522928,84.480764,57.185439,53.065463,23.693002,56.070461,47.251167,100.000000,93.463561,21.364695,Muy alto,0.334757
302,10,Durango,10014,Mezquital,48583,22.869092,57.154172,38.949868,42.404533,50.631232,50.820128,55.670326,100.000000,82.785739,28.181506,Muy alto,0.441567
946,18,Nayarit,18009,Del Nayar,47550,27.268016,66.476111,64.450424,32.724169,31.887197,47.535342,54.015839,100.000000,86.138199,28.628611,Muy alto,0.448573
599,14,Jalisco,14061,Mezquitic,22083,24.746515,63.047846,60.415671,30.726577,31.012900,46.076978,37.489361,100.000000,79.963235,31.264179,Muy alto,0.489869
451,12,Guerrero,12078,Cochoapa el Grande,21241,53.071253,82.818533,61.039822,9.957858,18.310526,43.647900,61.519905,100.000000,91.121495,32.164290,Muy alto,0.503972
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
963,19,Nuevo León,19006,Apodaca,656464,0.736270,11.873313,0.002763,0.014276,0.051578,0.334830,10.754394,2.701748,40.892191,61.274863,Muy bajo,0.960097
286,9,Ciudad de México,9015,Cuauhtémoc,545884,0.952548,13.528740,0.054147,0.029113,0.108849,0.122046,9.417070,0.000000,41.394890,61.287519,Muy bajo,0.960295
1003,19,Nuevo León,19046,San Nicolás de los Garza,412199,0.957916,15.005994,0.004859,0.017977,0.021136,0.480435,6.701794,0.000000,41.147158,61.377368,Muy bajo,0.961703
976,19,Nuevo León,19019,San Pedro Garza García,132169,1.015566,11.659507,0.003942,0.033893,0.037049,0.305738,6.202640,0.031021,32.817872,61.804519,Muy bajo,0.968396
